# Prediciendo la compra de audiolibros

En este ejemplo, lo que haremos es utilizar una red neuronal para predecir si una persona comprará o no un audiolibro. Para esto, usaremos un set de datos que contiene información de diferentes clientes, entre esta info podemos encontrar, la cantidad de veces que compro, la cantidad de veces que accedió al sitio, si dejo o no dejo una reseña, qué puntaje colocó en la reseña, entre otros.

Utilizaremos [Tensorflow](https://www.tensorflow.org/?gclid=Cj0KCQiAwJWdBhCYARIsAJc4idBKBFxY8qIb2YFIXTld4WhSSr7yb-b-0EUxf2CCWwKaXOUG44Wpt6IaAueKEALw_wcB) para crear un modelo de red neuronal.

### Importamos librerias

In [101]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample, shuffle
from sklearn.model_selection import train_test_split

import pickle

### Cargamos datos

In [38]:
raw_data = np.loadtxt('datasets/audiobooks_data.csv', delimiter=",")
raw_data[0,:].round(3)

array([8.730e+02, 2.160e+03, 2.160e+03, 1.013e+01, 1.013e+01, 0.000e+00,
       8.910e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00])

## Preprocesando datos

Antes que nada, vamos a preprocesar los datos.

#### Dataframe

Crearemos un dataframe con los datos cargados sólo para hacer un rápido análisis. Sin embargo, para entrenar y evaluar la red neuronal solo usaremos *raw_data*.

In [39]:
columnas = ["ID","Book length (mins)_overall","Book length (mins)_avg", "Price_overall",
            "Price_avg", "Review", "Review 10/10", "Completion", "Minutes Listened",
            "Support Requests", "Last visited minus Purchase date", "Targets"]

raw_df = pd.DataFrame(raw_data, columns=columnas)
# raw_df.describe().round(2).head()
raw_df.head()
# raw_df[raw_df["ID"]==994]

,ID,Book length (mins)_overall,Book length (mins)_avg,Price_overall,Price_avg,Review,Review 10/10,Completion,Minutes Listened,Support Requests,Last visited minus Purchase date,Targets
0,873.0,2160.0,2160.0,10.13,10.13,0.0,8.91,0.0,0.0,0.0,0.0,1.0
1,611.0,1404.0,2808.0,6.66,13.33,1.0,6.50,0.0,0.0,0.0,182.0,1.0
2,705.0,324.0,324.0,10.13,10.13,1.0,9.00,0.0,0.0,1.0,334.0,1.0
3,391.0,1620.0,1620.0,15.31,15.31,0.0,9.00,0.0,0.0,0.0,183.0,1.0
4,819.0,432.0,1296.0,7.11,21.33,1.0,9.00,0.0,0.0,0.0,0.0,1.0


In [40]:
raw_df.iloc[:,1:].describe().round(3)

,Book length (mins)_overall,Book length (mins)_avg,Price_overall,Price_avg,Review,Review 10/10,Completion,Minutes Listened,Support Requests,Last visited minus Purchase date,Targets
count,14084.000,14084.000,14084.000,14084.000,14084.000,14084.000,14084.000,14084.000,14084.000,14084.000,14084.000
mean,1591.282,1678.609,7.104,7.544,0.161,8.910,0.126,118.587,0.070,61.935,0.159
std,504.341,654.839,4.932,5.560,0.367,0.643,0.241,268.732,0.472,88.208,0.366
min,216.000,216.000,3.860,3.860,0.000,1.000,0.000,0.000,0.000,0.000,0.000
25%,1188.000,1188.000,5.330,5.330,0.000,8.910,0.000,0.000,0.000,0.000,0.000
50%,1620.000,1620.000,5.950,6.070,0.000,8.910,0.000,0.000,0.000,11.000,0.000
75%,2160.000,2160.000,8.000,8.000,0.000,8.910,0.130,64.800,0.000,105.000,0.000
max,2160.000,7020.000,130.940,130.940,1.000,10.000,1.000,2116.800,30.000,464.000,1.000


### Revisión del dataframe

Se explica brevemente qué representan algunas de las columnas.

- *Book length*: representa la duración en minutos de un libro.
- *Price overall*: precio pagado en dólares.
- *Review*: En el caso de que *Review* sea igual a 1, el cliente dió un puntaje a su compra. Este puntaje puede ir de 1 a 10.
- *Minutes listened*: indica la cantidad de tiempo que la persona escuchó el audiolibro.
- *Completion*: porcentaje que indica que tanto fue escuchado el libro respecto del total (en minutos).
- *Last visited minus Purchase date*: hace referencia a la cantidad de minutos que la persona entró al sitio desde su compra. Es esperable que mientras más grande sea este número, mayor sea la chance de que la persona compre.

Antes de explicar la última columna, se debe aclarar que el set de datos que estamos usando recopila información de clientes en un lapso de tiempo de 2 años y seis meses. Los primeros dos años forman las columnas que van desde *Book length (mins)_overall* hasta *Last visited minus Purchase date*. Los otros seis meses se utilizaron para formar la columna *Targets*.

- *Targets*: Esta columna contiene 0 y 1. Un cero indica que el cliente NO compro en el lapso de seis meses, y un uno indcia que sí lo hizo.

#### Completando valores cero

La columna *Review 10/10* posee muchos ceros. Esto es normal, en general ninguna persona deja un review. No obstante, debemos tomar alguna medida para no dejar tantos ceros en nuestros datos. Lo que vamos a hacer es rellenar dichos valores con el promedio de la columna, el cual es $8.91$.

In [41]:
raw_df[raw_df["Review 10/10"] == 0] = raw_df["Review 10/10"].mean()

### Balanceando el set de datos

Como siempre, debemos analizar los datos que tenemos para saber si están desbalanceados. Analicemos la columna *Targets* para ver que tan desbalanceado esta el set de datos considerando personas que sí compraron libros vs las que no han comprado.

In [42]:
raw_df["Targets"].sum()/raw_df["Targets"].shape[0]*100

15.88327179778472

Podemos ver que tenemos sólo un 15% de personas que hayan comprado libros (también vemos esto en la tabla *describe* del dataframe). ¿Por qué es esto importante? Porque un set de datos desbalanceado provocará que nuestra red neuronal (y cualquier otro algoritmo de ML) tenga un sesgo, ya que rápidamente interpretará que la clase importante es la clase 0, es decir, la gente que no compra libros.

Por lo tanto, vamos a balancear los datos. Pero antes, vamos a quedarnos con las columnas que harán las veces de *features* y la columna que hará de *target*.

Ahora vamos a balancear los datos. Lo que vamos a hacer es la técnica de *downsampling*, esto es, vamos a retirar algunas observaciones de la clase 0 (mayoritaria). Utilizaremos el método [Resample](https://scikit-learn.org/stable/modules/generated/sklearn.utils.resample.html) de Scikitlearn.

Info [acá](https://elitedatascience.com/imbalanced-classes).

In [43]:
clase_mayoritaria = shuffle(raw_df[raw_df["Targets"] == 0], random_state = 42) #mezclamos datos
clase_minoritaria = raw_df[raw_df["Targets"] == 1] #no hace falta mezclarlos

targets_uno = int(raw_df[raw_df["Targets"] == 1]["Targets"].sum()) #cantidad de targets con valor 1

# Aplicamos el downsampling
clase_mayoritaria_downsampled = resample(clase_mayoritaria, 
                                 replace=False,    # Sin reemplazo
                                 n_samples = targets_uno,     # Cantidad de muestras que queremos.
                                 random_state=42) # Seteamos la semilla para tener reproducibilidad en un futuro

df_balanceado = pd.concat([clase_mayoritaria_downsampled,clase_minoritaria])
df_balanceado["Targets"].value_counts()

0.0    2237
1.0    2237
Name: Targets, dtype: int64

Podemos ver que ahora el set de datos está balanceado. Lo malo es que hemos perdido muestras, pero es el costo que tenemos que pagar.

#### Separando variables independientes y variable dependiente

In [82]:
features = ["Book length (mins)_overall","Book length (mins)_avg", "Price_overall", "Price_avg", "Review", "Review 10/10",
            "Completion", "Minutes Listened", "Support Requests", "Last visited minus Purchase date"]

inputData = df_balanceado[features].values #no usamos la columna ID ya que no aporta nada
# inputData = df_balanceado[:,1:-1] #forma equivalente

targets = df_balanceado["Targets"].values

### Estandarizando datos

Como hemos mencionado, es importante dentro del preprocesamiento hacer que los datos estén estandarizados, es decir, lograr que sus valores máximos y mínimos estén en un rango acotado y al mismo tiempo lograr que tengan media cero y desvío estándar 1 (o cercano a estos valores). Usaremos el método [standardscaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn-preprocessing-standardscaler).

In [83]:
scaler = StandardScaler()
inputData_std = scaler.fit_transform(inputData)
inputData_std = pd.DataFrame(inputData_std, columns = ["Book length (mins)_overall","Book length (mins)_avg",
                                                        "Price_overall", "Price_avg", "Review", "Review 10/10",
                                                        "Completion", "Minutes Listened", "Support Requests",
                                                        "Last visited minus Purchase date"])
inputData_std.describe().round(2)

,Book length (mins)_overall,Book length (mins)_avg,Price_overall,Price_avg,Review,Review 10/10,Completion,Minutes Listened,Support Requests,Last visited minus Purchase date
count,4474.00,4474.00,4474.00,4474.00,4474.00,4474.00,4474.00,4474.00,4474.00,4474.00
mean,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00
std,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
min,-2.69,-1.85,-0.63,-0.69,-0.44,-11.83,-0.39,-0.45,-0.20,-0.77
25%,-0.74,-0.74,-0.35,-0.47,-0.44,-0.01,-0.39,-0.45,-0.20,-0.77
50%,0.13,-0.24,-0.23,-0.27,-0.44,-0.01,-0.39,-0.45,-0.20,-0.55
75%,1.21,0.37,0.15,0.04,-0.44,-0.01,-0.39,-0.19,-0.20,0.65
max,1.21,5.94,18.16,14.92,2.27,1.61,4.64,7.81,19.42,3.43


### Separando datos en set de entrenamiento, validación y testeo

In [95]:
tarin = 0.8
validation = 0.1
test = 0.1

#Primero separo en set de entrenamiento y de testeo
x_train, x_temp, y_train, y_temp = train_test_split(inputData_std.values, targets, test_size = 1 - tarin)

#Ahora separo el set de testeo en sets de validación y de testeo
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size = test/(test + validation)) 


#### Guardando los sets y el escalor

In [103]:
# Descomentar las lineas debajo para guardar los sets

# np.savez('datos_entrenamiento', inputs = x_train, target = y_train)
# np.savez('datos_validación', inputs = x_val, targets = y_val)
# np.savez('datos_testeo', inputs = x_test, targets = y_test)

# pickle.dump(scaler, open("scaler.pickle", "wb"))